In [1]:
import gzip
from struct import unpack, pack, calcsize

In [4]:
# Offset + length_last - 1
# These values are taken from 
message_type_size_dict = {
    "S" : 11,
    "R" : 38,
    "H" : 24,
    "Y" : 19,
    "L" : 25,
    "V" : 34,
    "W" : 11,
    "K" : 27,
    "J" : 34,
    "h" : 20,
    "A" : 35,
    "F" : 39,
    "E" : 30,
    "C" : 35,
    "X" : 22,
    "D" : 18,
    "U" : 34,
    "P" : 43,
    "Q" : 39,
    "B" : 18,
    "I" : 49,
    "N" : 19,
    "\x00" : 1
}

In [2]:
def handle_system_messages():
    # System Message
    #stock_locate, tracking_number, time_stamp1, time_stamp2, event_code = unpack('>HHhLc', msg);
    #time_stamp = time_stamp1 | (time_stamp2 << 16) # Merging timestamp
    stock_locate, tracking_number, time_stamp, event_code = unpack('>HH6cc', msg);
    global market_o, market_s, market_q, market_m, market_e, market_c
    if event_code == "O":
        market_o = market_o + 1
        # Start of Messages.Outside oftime stamp messages,the start of day message isthe first message sentin
        # any trading day
        print stock_locate, tracking_number, time_stamp, event_code
    elif event_code == "S":
        market_s = market_s + 1
        # Start of System hours. This message indicatesthatNASDAQis open and ready to start accepting orders
        print stock_locate, tracking_number, time_stamp, event_code
    elif event_code == "Q":
        market_q = market_q + 1
        # Start of Market hours. This message isintended to indicate that Market Hours orders are available
        # for execution
        print stock_locate, tracking_number, time_stamp, event_code
    elif event_code == "M":
        market_m = market_m + 1
        # End of Market hours. This message isintended to indicate that Market Hours orders are no longer
        # available for execution
        print stock_locate, tracking_number, time_stamp, event_code
    elif event_code == "E":
        market_e = market_e + 1
        # End of System hours. It indicates that Nasdaq is now closed and will not accept any new orderstoday.
        # It is still possible to receive Broken Trade messages and Order Delete messages after the End of Day
        print stock_locate, tracking_number, time_stamp, event_code
    elif event_code == "C":
        market_c = market_c + 1
        # End of Messages. This is always the last message sent in any trading day.
        print stock_locate, tracking_number, time_stamp, event_code

In [ ]:
bin_data = file("20190130.PSX_ITCH_50", "rb")
message_type = bin_data.read(1);
market_o, market_s, market_q, market_m, market_e, market_c = (0, 0, 0, 0, 0, 0)
number_of_messages = 0
while message_type:
    msg = bin_data.read(message_type_size_dict[message_type])
    number_of_messages += 1
    # trading Message
    if message_type == "H":
        temp = unpack('>HHhL8ccc4c', msg);
    elif message_type == "S":
        handle_system_messages()
    message_type = bin_data.read(1);
    if number_of_messages > 100000000: # for all change to 100000000 or remove this if and break!
        break
    

print number_of_messages, market_o, market_s, market_q, market_m, market_e, market_c

0 0 68096397543979 O
0 0 147082811087409 S
0 0 227440503103258 Q


In [ ]:
calcsize('>HHhL8ccc4c')
calcsize('>HHhLc')
calcsize('HHI')

# Making a 6 byte timestamp
def unpack48(x):
    x1, x2, x3 = struct.unpack('<HHI', x)
    return x1, x2 | (x3 << 16)